# torch.nn           
torch.nn是专门为深度学习而设计的模块。torch.nn的核心数据结构是`Module`，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。   
在实际使用中，最常见的做法是继承`nn.Module`，撰写自己的网络/层。

## 1.1 自定义层              
如何用nn.Module实现自己的全连接层。全连接层，又名仿射层，输出$\textbf{y}$和输入$\textbf{x}$满足$\textbf{y=Wx+b}$，$\textbf{W}$和$\textbf{b}$是可学习的参数

In [0]:
import torch as t
from torch import nn

In [0]:
class Linear(nn.Module):
    def __init__(self, in_f, out_f):
        # 等价于 nn.Module.__init(self)
        super(Linear, self).__init__()
        self.w = nn.Parameter(t.randn(in_f, out_f))
        self.b = nn.Parameter(t.randn(out_f))
        
    def forward(self, x):
        x = x.mm(self.w)
        return x + self.b.expand_as(x)

In [3]:
layer = Linear(4, 3)
input = t.randn(2, 4)
output = layer(input)
output

tensor([[-6.1545, -2.6879, -4.4222],
        [ 0.8841, -2.5078, -2.4721]], grad_fn=<AddBackward0>)

In [4]:
for name, parameter in layer.named_parameters():
    print(name, parameter)

w Parameter containing:
tensor([[-1.3030,  0.5937,  0.0578],
        [ 0.0861, -0.2141, -1.4550],
        [ 1.5023,  0.2462, -0.1321],
        [-0.6188, -0.6013, -1.1792]], requires_grad=True)
b Parameter containing:
tensor([-1.7979, -1.4262,  0.0190], requires_grad=True)


### 在自定义层时，需注意以下几点：
- 自定义层`Linear`必须继承`nn.Module`，并且在其构造函数中需调用`nn.Module`的构造函数，即`super(Linear, self).__init__()` 或`nn.Module.__init__(self)`，推荐使用第一种用法，尽管第二种写法更直观。
- 在构造函数`__init__`中必须自己定义可学习的参数，并封装成`Parameter`，如在本例中我们把`w`和`b`封装成`parameter`。`parameter`是一种特殊的`Tensor`，但其默认需要求导（requires_grad = True），感兴趣的读者可以通过`nn.Parameter??`，查看`Parameter`类的源代码。
- `forward`函数实现前向传播过程，其输入可以是一个或多个tensor。
- 无需写反向传播函数，nn.Module能够利用autograd自动实现反向传播，这点比Function简单许多。
- 使用时，直观上可将layer看成数学概念中的函数，调用layer(input)即可得到input对应的结果。它等价于`layers.__call__(input)`，在`__call__`函数中，主要调用的是 `layer.forward(x)`，另外还对钩子做了一些处理。所以在实际使用中应尽量使用`layer(x)`而不是使用`layer.forward(x)`
- `Module`中的可学习参数可以通过`named_parameters()`或者`parameters()`返回迭代器，前者会给每个parameter都附上名字，使其更具有辨识度。

可见利用Module实现的全连接层，比利用`Function`实现的更为简单，因其不再需要写反向传播函数

## 1.2 多层感知机      
Module能够自动检测到自己的`Parameter`，并将其作为学习参数。除了`parameter`之外，Module还包含子`Module`，主Module能够递归查找子`Module`中的`parameter`。    
下面再来看看稍微复杂一点的网络，多层感知机。

In [0]:
class Perceptron(nn.Module):
    def __init__(self, in_f, hidden_f, out_f):
        nn.Module.__init__(self)
        # 利用前面定义的全连接层
        self.layer1 = Linear(in_f, hidden_f)
        self.layer2 = Linear(hidden_f, out_f)
        
    def forward(self, x):
        x = self.layer1(x)
        x = t.sigmoid(x)
        return self.layer2(x)

In [12]:
perceptron = Perceptron(3, 4, 1)
input = t.randn(4, 3)
out = perceptron(input)
out

tensor([[ 0.6573],
        [ 1.4813],
        [-0.5200],
        [ 2.0085]], grad_fn=<AddBackward0>)

In [13]:
for name, param in perceptron.named_parameters():
    print(name, param, param.size())

layer1.w Parameter containing:
tensor([[ 0.6701,  1.1535, -0.0547, -0.8057],
        [-1.6345,  0.2124,  1.8351, -1.8436],
        [-0.1596,  0.0630, -0.3726,  0.7524]], requires_grad=True) torch.Size([3, 4])
layer1.b Parameter containing:
tensor([-1.0419, -0.2923,  1.3164,  0.7752], requires_grad=True) torch.Size([4])
layer2.w Parameter containing:
tensor([[-0.9762],
        [-1.5965],
        [ 1.0703],
        [ 0.4736]], requires_grad=True) torch.Size([4, 1])
layer2.b Parameter containing:
tensor([0.8269], requires_grad=True) torch.Size([1])


可见，即使是稍复杂的多层感知机，其实现依旧很简单。 构造函数`__init__`中，可利用前面自定义的Linear层(module)，作为当前module对象的一个子module，它的可学习参数，也会成为当前module的可学习参数。

### module中parameter的命名规范：
- 对于类似`self.param_name = nn.Parameter(t.randn(3, 4))`，命名为`param_name`
- 对于子Module中的parameter，会其名字之前加上当前Module的名字。如对于`self.sub_module = SubModel()`，SubModel中有个parameter的名字叫做param_name，那么二者拼接而成的parameter name 就是`sub_module.param_name`。

### PS：查看关于layer文档  
为方便用户使用，PyTorch实现了神经网络中绝大多数的layer，这些layer都继承于nn.Module，封装了可学习参数`parameter`，并实现了forward函数，且很多都专门针对GPU运算进行了CuDNN优化，其速度和性能都十分优异。       

对nn.Module中的所有层的具体内容，读者可参照官方文档或在IPython/Jupyter中使用nn.layer?来查看。阅读文档时应主要关注以下几点：

- 构造函数的参数，如nn.Linear(in_features, out_features, bias)，需关注这三个参数的作用。
- 属性、可学习参数和子module。如nn.Linear中有`weight`和`bias`两个可学习参数，不包含子module。
- 输入输出的形状，如nn.linear的输入形状是(N, input_features)，输出为(N，output_features)，N是batch_size。

这些自定义layer对输入形状都有假设：输入的不是单个数据，而是一个batch。输入只有一个数据，则必须调用`tensor.unsqueeze(0)` 或 `tensor[None]`将数据伪装成batch_size=1的batch

## 1.3 nn.functional       
也是nn中常用的模块，nn中的大多数layer，在`functional`中都有一个与之相对应的函数。   

`nn.functional`中的函数和`nn.Module`的主要区别：   
- 用nn.Module实现的layers是一个特殊的类，都是由`class layer(nn.Module)`定义，会自动提取可学习的参数   
- `nn.functional`中的函数更像是纯函数，由`def function(input)`定义    

下面举例说明functional的使用，并指出二者的不同之处

In [7]:
input = t.randn(2, 3)
model = nn.Linear(3, 4)
output1 = model(input)
output2 = nn.functional.linear(input, model.weight, model.bias)
output1, output2

(tensor([[-0.4439, -0.2176,  0.3737, -0.9774],
         [-0.5911, -0.1468,  0.5085, -0.7642]], grad_fn=<AddmmBackward>),
 tensor([[-0.4439, -0.2176,  0.3737, -0.9774],
         [-0.5911, -0.1468,  0.5085, -0.7642]], grad_fn=<AddmmBackward>))

### 如何选择`nn.Module`和`nn.functional`     
模型有可学习的参数，最好用nn.Module，否则既可以使用nn.functional也可以使用nn.Module，二者在性能上没有太大差异，具体的使用取决于个人的喜好。      

如激活函数（ReLU、sigmoid、tanh），池化（MaxPool）等层由于没有可学习参数，则可以使用对应的functional函数代替，而对于卷积、全连接等具有可学习参数的网络建议使用nn.Module    

**Dropout**    
虽然dropout操作也没有可学习操作，但建议还是使用`nn.Dropout`而不是`nn.functional.dropout`，因为dropout在训练和测试两个阶段的行为有所差别，使用`nn.Module`对象能够通过`model.eval`操作加以区分

In [0]:
from torch.nn import functional as f

class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)